This notebook presents a BiLSTM model to generate chords giving a leadsheet.


In [0]:
import os
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import trange

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Change to the adequate directory
os.chdir(os.path.join("drive", "My Drive", "Projet_rendu", "BiLSTM"))

In [0]:
def data(batch_size):
    """
    New format of input and target vectors

    :param batch_size:
    :return:
    Process data for torch
    For crossentropy, target must be a number (from 0 to number of classes)


    measures : padded to 184/song (max measure of in a song in dataset)
    notes : padded to 32/ measure
        => Suppression des mesures rajoutées au padding (vides). Les mesures paddées ont un chord de 0 correspondant
    :param batch_size:
    :return: train , validation and test dataloader
    """
    # Loading our data (already padded)
    # nbr_song * nbr_measures_kept_by_song * nbr_notes_by_measure * onehot_note
    train_inputs = np.load('Données/train_input.npy', allow_pickle=True)
    # nbr_songs * nbr_measures_kept * onehot_chord
    train_targets = np.load('Données/train_target.npy', allow_pickle=True)
    test_inputs = np.load('Données/test_input.npy', allow_pickle=True)
    test_targets = np.load('Données/test_target.npy', allow_pickle=True)

    train_targets = np.array([np.argmax(i) for i in train_targets])  # index => for crossentropy loss
    test_targets = np.array([np.argmax(i) for i in test_targets])

    train_inputs, validation_inputs, train_targets, validation_targets = train_test_split(train_inputs, train_targets,
                                                                                          random_state=2018,
                                                                                          test_size=0.1,
                                                                                          shuffle=True)

    # Convert all of our data into torch tensors, the required datatype for our modele
    train_inputs = torch.tensor(train_inputs, dtype=torch.float)  # float
    validation_inputs = torch.tensor(validation_inputs, dtype=torch.float)
    train_targets = torch.tensor(train_targets, dtype=torch.long)  # long: classification
    validation_targets = torch.tensor(validation_targets, dtype=torch.long)
    test_inputs = torch.tensor(test_inputs, dtype=torch.float)
    test_targets = torch.tensor(test_targets, dtype=torch.long)

    # Create an iterator of our data with torch DataLoader for memory efficency.
    train_data = TensorDataset(train_inputs, train_targets)
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    validation_data = TensorDataset(validation_inputs, validation_targets)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

    test_data = TensorDataset(test_inputs, test_targets)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    return train_dataloader, validation_dataloader, test_dataloader  # TODO num workers CPU + pin_memory = True




In [0]:
class BiLSTM(nn.Module):
    """
    input => 1 measure/bar = sequence of 32 notes (onehot size 12)
    output: chord, single value (=index in onehot chord vector sized 24 in target)

    Achieve 0.51 accuracy on validation set, 0.463 on test
    # TODO: masking padding values, batch normalization, confusion matrix, testset, patience=10, other lstm parameters...
    """

    def __init__(self, dropout=0.3, hidden_size=256, num_layer=2, input_size=12, bidirectional=True):
        """

        :param dropout: dropout rate between each of the layer (2) of the LSTM.
        only our hidden state from the first layer is passed as input to the second layer, and not the cell state
        :param hidden_size:
        :param num_layer:
        :param input_size: note_dim, 12 , sequence de 32 vecteurs.
        Le lstm retourne un seul input (pas besoin d'un input par entrée de la séquence), de taille hidden_size
        """

        super().__init__()
        self.bidirectional=bidirectional
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layer,
                            dropout=dropout,
                            bidirectional=bidirectional,
                            batch_first=True)
        self.fc = nn.Linear(in_features=2 * hidden_size, out_features=24)

    def forward(self, x):
        """
        entier sequence at once, only one measure as input so one chord at output, no
        need to generate one chord every measure like in a sequence of measure

        hidden state and cell state: in torch by default initialized to 0
        lstm: every hidden state produce output. Keep last hidden state only
        Bidirectional: hiddenstate dim *2

        Keep only output from last hidden state of the LSTM that correspond to the representation of the entire sequence
        (each output is size 256=2*hidden_size, 32 output because sequence of length 32)
        When using a BiLSTM the hidden states of the directions are just concatenated
        :param x:
        :return:
        """
        outputs, (hidden, cell) = self.lstm(x)  # (batch_size, seq_len, 2*hidden_size)
        output = outputs[:, -1, :]  # (1024, 256). -1: pick the last hidden state (last element of the seq)

        output = self.fc(output)  # (batch_size, 24)
        output = F.log_softmax(output)  # si loss pas crossentropy, sinon inutile
        return output  # sans F.logsoftmax si crossentropy car logsoftmax computed directly in crossentropy loss

In [0]:
def accuracy(outputs, targets):
    """
    :param outputs: (batch_size, 24) from model where 24 sized vector is probas (logits) for one chord
    :param targets: (batch_size) , each chord is a number meaning its class
    :return: accuracy for one batch
    # TODO use torch.max(tensor, 1 ou 0) pour indice et value tjrs dans le gpu
    """
    batch_size = targets.shape[0]
    # log croissant donc log:softmax => valeur désirée est la plus grande (plus grande proba)
    idx_predicted = np.argmax(outputs, 1)
    return (idx_predicted == targets).sum() / batch_size

In [0]:
def train(model, train_dataloader, validation_dataloader, learning_rate=0.001, eps=1e-8, epochs=400, patience=15,
          model_name="BILSTM.pth"):
    """
    target: must be a single number, not a one hot... for using cross_entropy
    :param patience: break if no improvment of validation accuracy after waiting s10 epochs
    :param eps:
    :param model:
    :param batch_size:
    :param learning_rate:
    :param epochs:
    :return:
    """
    # device_name = tf.test.gpu_device_name()
    # if device_name != '/device:GPU:0':
    #     raise SystemError('GPU device not found')
    # print('Found GPU at: {}'.format(device_name))
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    best_loss = 100
    best_model = None

    num_training_steps = (len(train_dataloader) * epochs)  # len(train_dataloader) = len(train_data)// batch_size

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, eps=eps)
    # PyTorch scheduler: reduce lr by factor after patience with no improvment of val_loss (mieux que StepLR)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=False, threshold=0.0001,
                                  threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
    # criterion = nn.CrossEntropyLoss()  # Because classification pb (if regression, MSE). = nn.logsoftmax + nn.NLLLoss
    criterion = nn.NLLLoss()  # si nn.logsoftmax dans le modele (pour le test)

    max_grad_norm = 1.0  # Gradient threshold, gradients norms that exceed this threshold are scaled down to match
    # the norm.

    # Tracking values
    val_accuracies = list()
    val_losses = list()
    train_accuracies = list()
    train_losses = list()

    p = 0  # patience

    model.cuda()
    for epoch in trange(epochs, desc="Epoch"):
        if p >= patience:
            break

        model.zero_grad()  # Clear stored gradient
        model.train()  # training mode

        # Tracking variables
        tr_loss, tr_acc, nb_tr_steps = 0, 0, 0

        # Train on all our data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Add batch to GPU
            b_inputs, b_targets = tuple(t.to(device) for t in batch)

            # Forward pass
            outputs = model(b_inputs)

            # loss function (negative log likelihood because of the last log softmax layer) = multiclass cross-entropy
            # "size_average" is set to "False", the losses are summed for each minibatch.
            loss = criterion(outputs, b_targets)

            # Backward pass
            loss.backward()

            # Update tracking variables
            tr_loss += loss.item()

            # Move logits and labels to CPU
            outputs = outputs.detach().cpu().numpy()
            b_targets = b_targets.to('cpu').numpy()
            tr_acc += accuracy(outputs, b_targets)

            # AdamW optimizer: update parameters and take a step using the computed gradient
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()  # Perform a step of gradient descent
            model.zero_grad()  # Reset gradient

            nb_tr_steps += 1
            # if nb_tr_steps % 50 == 0:
            #     print("Step {} out  of {} /n".format(nb_tr_steps, (num_training_steps // epochs)))

        train_accuracy = tr_acc / nb_tr_steps
        train_loss = tr_loss / nb_tr_steps
        train_accuracies.append(train_accuracy)
        train_losses.append(train_loss)

        # 2. Validation step
        # Put model in evaluation mode to evaluate loss on the validation set
        model.eval()

        # Tracking variables
        nb_eval_steps, v_acc, v_loss = 0, 0, 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:
            # Add batch to GPU, Unpack the inputs from our dataloader
            b_inputs, b_targets = tuple(t.to(device) for t in batch)
            # Telling the model not to compute or store gradients, saving memory and speeding up validation
            with torch.no_grad():
                # Forward pass, calculate logit predictions
                outputs = model(b_inputs)

            v_loss += criterion(outputs, b_targets).item()

            # Move logits and labels to CPU
            outputs = outputs.detach().cpu().numpy()
            b_targets = b_targets.to('cpu').numpy()

            v_acc += accuracy(outputs, b_targets)
            nb_eval_steps += 1
        val_accuracy = v_acc / nb_eval_steps
        val_accuracies.append(val_accuracy)

        val_loss = v_loss / nb_eval_steps
        val_losses.append(val_loss)

        # avoid overfitting
        if val_loss < best_loss:
            p = 0
            best_model = model
            best_loss = val_loss
            torch.save(best_model.state_dict(), os.path.join(os.getcwd(), "Modèles entraînés", "BILSTM_" + model_name))
        else:
            p += 1

        print("Train loss: {} - Train accuracy: {} -  Val loss: {} - Val accuracy: {}".format(train_loss, train_accuracy
                                                                                              , val_loss, val_accuracy))

        scheduler.step(val_loss)

    return best_model, train_losses, train_accuracies, val_accuracies, val_losses

In [0]:
def test(model, test_dataloader, show_results=False):  # TODO sortir ouput pour une chanson avec index chanson
    """
    Test model on test songs
    :param mode: type of vector/feature to keep
    :param batch_size:
    :param model:
    :return:
    """

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    test_acc = 0
    nb_tr_steps = 0
    with torch.no_grad():
        for step, batch in enumerate(test_dataloader):
            # Add batch to GPU
            b_inputs, b_targets = tuple(t.to(device) for t in batch)
            outputs = model(b_inputs)

            outputs = outputs.detach().cpu().numpy()
            b_targets = b_targets.to('cpu').numpy()
            
            if show_results:
              chords = np.argmax(outputs, 1)
              print([chord_dictionary[chord] for chord in chords])

            test_acc += accuracy(outputs, b_targets)
            nb_tr_steps += 1
        test_accuracy = test_acc / nb_tr_steps
    print("Test accuracy: {}".format(test_accuracy))
    pass


In [0]:
def show_result(train_losses, train_accuracies, val_accuracies, val_losses):
    plt.plot(np.arange(len(val_accuracies)), train_losses, label='train loss')
    plt.plot(np.arange(len(val_accuracies)), val_losses, label='val loss')
    plt.legend(loc="upper right")
    plt.title("Negative log-likelihood val-train")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(np.arange(len(val_accuracies)), val_accuracies, label='val accuracy')
    plt.plot(np.arange(len(val_accuracies)), train_accuracies, label='train accuracy')
    plt.legend()
    plt.title("Accuracy over validation/train-set")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()
    pass

In [0]:
  train_dataloader, validation_dataloader, test_dataloader = data(batch_size=2048)

In [0]:
  model = BiLSTM()
  best_model, train_losses, train_accuracies, val_accuracies, val_losses = train(model, train_dataloader,
                                                                                   validation_dataloader,
                                                                                   model_name="NAME",
                                                                                   learning_rate=0.001)
print("DONE")

Epoch:   0%|          | 1/400 [00:10<1:11:23, 10.73s/it]

Train loss: 2.595307633280754 - Train accuracy: 0.2709981944304912 -  Val loss: 2.2558361291885376 - Val accuracy: 0.3065909264984403


Epoch:   0%|          | 2/400 [00:21<1:11:08, 10.73s/it]

Train loss: 2.2373235523700714 - Train accuracy: 0.29927323842012576 -  Val loss: 2.222332000732422 - Val accuracy: 0.3065909264984403


Epoch:   1%|          | 3/400 [00:32<1:10:59, 10.73s/it]

...

Train loss: 1.3049456775188446 - Train accuracy: 0.5927203664620958 -  Val loss: 1.6258863806724548 - Val accuracy: 0.5211449716605392


Epoch:  24%|██▍       | 96/400 [17:45<56:39, 11.18s/it]

Train loss: 1.2970091551542282 - Train accuracy: 0.5941109015252695 -  Val loss: 1.6258520483970642 - Val accuracy: 0.5212670419730392


Epoch:  24%|██▍       | 97/400 [17:56<56:26, 11.18s/it]

Train loss: 1.2994388565421104 - Train accuracy: 0.5927686359445915 -  Val loss: 1.6258712410926819 - Val accuracy: 0.5212670419730392


In [0]:
  # Si la classe du modele change, on aura la nouvelle classe loadée
model_trained = BiLSTM()
model_trained.load_state_dict(torch.load(os.path.join(os.getcwd(), "Modèles entraînés", "BILSTM.pth")))
test(model_trained, test_dataloader, show_results=False)